<a href="https://colab.research.google.com/github/farz1313/Farz-Projects/blob/main/nlp_topic_mallet_27_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en
#conda install -c conda-forge spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 28.8 MB/s 
     |████████████████████████████████| 15.7 MB 183 kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=203f37cfe437f8f96b5c9e0453749834098adac624d222bf1324b85af4195fa7
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [2]:
!pip install gensim

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['product', 'product_band', 'www'])

In [6]:
# Import Dataset
df = pd.read_csv('fb_ads.csv') #, encoding='cp1252')

df.head()

,ad_id,page_id,page_name,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,funding_entity,impressions_min,impressions_max,spend_min,spend_max,ad_url
0,422855775441062,1787487104836976,Plasnewydd & Roath Community News,If you live in Plasnewydd & you're self-isolat...,NaN,NaN,NaN,2021-02-15,NaN,Plasnewydd & Roath Community News,0,999,0,99,https://www.facebook.com/ads/library/?id=42285...
1,129870885665778,101558386889,Shelter Scotland,Shelter Scotland's Law & Debt advice project c...,blog.scotland.shelter.org.uk,NaN,Shelter Scotland,2021-02-15,NaN,Shelter Scotland,0,999,0,99,https://www.facebook.com/ads/library/?id=12987...
2,414306946330402,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=41430...
3,1123586888080157,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=11235...
4,1374708909543169,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=13747...


In [7]:
# Convert to list

df['ad_creative_body_str']=df['ad_creative_body'].apply(str)


In [8]:
data = df.ad_creative_body_str.values.tolist()

In [10]:
data

["If you live in Plasnewydd & you're self-isolating or struggling to get out, your local Councillors are here to help. Just email contact@plasnewyddlabour.com",
 "Shelter Scotland's Law & Debt advice project can help if you are worried about paying your rent or your mortgage. Or even if you are at risk of eviction.\n\nIf you live in the court areas of: Perth, Dundee, Jedburgh, Selkirk, Forfar, Alloa, Falkirk & Stirling, the project can provide free and confidential debt and legal advice & even legal representation if you need it.",
 '{{product.brand}}',
 '{{product.brand}}',
 '{{product.brand}}',
 '{{product.brand}}',
 "Open up the world for your children with The Week Junior. It's packed with safe, inspiring content & fun, absorbing activities",
 "Open up the world for your children with The Week Junior. It's packed with safe, inspiring content & fun, absorbing activities",
 "Open up the world for your children with The Week Junior. It's packed with safe, inspiring content & fun, abso

In [11]:
#str(df['ad_creative_body'])

In [12]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['If you live in Plasnewydd & youre self-isolating or struggling to get out, your local Councillors are here to help. Just email ']


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['if', 'you', 'live', 'in', 'plasnewydd', 'youre', 'self', 'isolating', 'or', 'struggling', 'to', 'get', 'out', 'your', 'local', 'councillors', 'are', 'here', 'to', 'help', 'just', 'email']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['if', 'you', 'live', 'in', 'plasnewydd', 'youre', 'self', 'isolating', 'or', 'struggling', 'to', 'get', 'out', 'your', 'local', 'councillors', 'are', 'here', 'to', 'help', 'just', 'email']


In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en

nlp = spacy.load('en_core_web_sm')

#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['live', 'be', 'self', 'isolate', 'struggle', 'get', 'local', 'councillor', 'help', 'email']]


In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [18]:
id2word[0]

'be'

In [19]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('be', 1),
  ('councillor', 1),
  ('email', 1),
  ('get', 1),
  ('help', 1),
  ('isolate', 1),
  ('live', 1),
  ('local', 1),
  ('self', 1),
  ('struggle', 1)]]

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.439*"get" + 0.106*"change" + 0.062*"sign" + 0.056*"generate" + '
  '0.037*"electricity" + 0.033*"tackling_climate" + 0.029*"minute" + '
  '0.023*"independent_generator" + 0.020*"making_difference" + '
  '0.017*"worst_impact"'),
 (1,
  '0.256*"know" + 0.184*"guide" + 0.121*"home" + 0.070*"could" + 0.043*"pay" + '
  '0.036*"thousand" + 0.028*"save" + 0.014*"average" + 0.010*"stop" + '
  '0.009*"everyone_reading"'),
 (2,
  '0.145*"day" + 0.134*"worried" + 0.082*"switch" + 0.068*"brand" + '
  '0.065*"new" + 0.035*"send" + 0.029*"thank" + 0.025*"less" + 0.024*"street" '
  '+ 0.021*"invest"'),
 (3,
  '0.097*"good" + 0.096*"energy" + 0.094*"people" + 0.061*"year" + '
  '0.053*"detail" + 0.052*"business" + 0.052*"importance" + 0.052*"issue" + '
  '0.051*"support" + 0.033*"work"'),
 (4,
  '0.354*"today" + 0.227*"free" + 0.126*"help" + 0.033*"believe" + '
  '0.024*"area" + 0.023*"provide" + 0.023*"even" + 0.017*"risk" + '
  '0.014*"demand" + 0.014*"law"'),
 (5,
  '0.133*"tool" + 0.063*

In [22]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [23]:
get_lda_topics(lda_model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,get,know,day,good,today,tool,exploitation,take,use,staff
1,change,guide,worried,energy,free,name,contextual_approach,end,request,planet
2,sign,home,switch,people,help,come,young,second,renewable,green
3,generate,could,brand,year,believe,story,reframe,sustainability,supply,important
4,electricity,pay,new,detail,area,claim,inbox,store_worker,code,avoid
5,tackling_climate,thousand,send,business,provide,step,weve_partnered,compare,stand,speak
6,minute,save,thank,importance,even,independent,free_webinar,sustainability_seriously,threat,match
7,independent_generator,average,less,issue,risk,invite,create,complete,urgent_sm,gift
8,making_difference,stop,street,support,demand,certain,confident,cage,reduce,make
9,worst_impact,everyone_reading,invest,work,law,sure,models_child,nightmare,international,safe


In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.387250846373098

Coherence Score:  0.48062296031425084


In [30]:
import pyLDAvis.gensim_models as gensimvis

In [31]:

#import pyLDAvis.gensim
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.227691 -0.060338       1        1  18.175254
3     -0.211332 -0.056541       2        1  13.375307
5      0.055902 -0.012736       3        1   6.700529
2      0.019934 -0.075869       4        1   5.777594
4     -0.074091  0.351751       5        1   5.412658
1      0.039018 -0.022569       6        1   4.254965
15     0.003434 -0.015137       7        1   4.133649
8      0.013702 -0.102833       8        1   4.122173
13     0.013641 -0.079137       9        1   4.103178
17     0.057269 -0.028296      10        1   4.031574
0     -0.029466 -0.018992      11        1   3.761396
14     0.072474 -0.025175      12        1   3.559289
9      0.021947 -0.024217      13        1   3.261038
18    -0.006150  0.285823      14        1   3.077073
6     -0.287286 -0.058798      15        1   3.037319
7      0.024475 -0.015252      16        1   2.824766
11     0.146395 -0.009613      17        1   2.709500
12     0.107842 -0.010980      18        1   2.588388
10     0.138715 -0.010045      19        1   2.556391
16     0.121265 -0.011044      20        1   2.537960, topic_info=            Term         Freq        Total Category  logprob  loglift
78         today  1528.000000  1528.000000  Default  30.0000  30.0000
3            get  1318.000000  1318.000000  Default  29.0000  29.0000
238         take  1041.000000  1041.000000  Default  28.0000  28.0000
415         join  1023.000000  1023.000000  Default  27.0000  27.0000
34         child   898.000000   898.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
5474        surf    16.647878    17.339179  Topic20  -4.8014   3.6331
5475       whore    16.647878    17.339179  Topic20  -4.8014   3.6331
5472  finisterre    16.645495    17.336905  Topic20  -4.8016   3.6331
3620      legacy    14.717383    15.399945  Topic20  -4.9247   3.6285
3885   injustice    14.520956    15.203518  Topic20  -4.9381   3.6279

[645 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
5466     12  0.987920         ability
915      10  0.999360            able
4611     13  0.951818          abuser
892       4  0.981130     accountable
1333     13  0.985777         achieve
...     ...       ...             ...
224       2  0.984859           would
225       2  0.998954            year
926      15  0.994626           young
5520     11  0.974085  youve_actually
5333      9  0.964527    youve_worked

[615 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 4, 6, 3, 5, 2, 16, 9, 14, 18, 1, 15, 10, 19, 7, 8, 12, 13, 11, 17])

In [32]:
vis = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'output_lda_model.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
#conda install -c conda-forge mallet

In [33]:
#import subprocess
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
#mallet_path = 'path/to/mallet-2.0.8/bin/mallet' # update this path
#mallet_path = "C:\Users\farhat\OneDrive\Documents\python\NLP project\data\mallet-2.0.8\bin\mallet" # update this path
#mallet_path = "C:\\Users\\farhat\\OneDrive\\Documents\\python\\NLP project\\data\\mallet-2.0.8\\bin\\mallet" # update this path
#mallet_path = '/Users/kofola/Downloads/mallet-2.0.7/bin/mallet'
#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

#mallet_path = r'D:\mallet-2.0.8\bin\mallet'
import os
os.environ.update({'MALLET_HOME':r'D:/mallet-2.0.8/'})
#mallet_path = r'C:\Users\farhat\OneDrive\Documents\python\NLP project\data\mallet-2.0.8\bin\mallet'
mallet_path = r'D:\mallet-2.0.8\bin\mallet'


ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
#model = models.LdaMallet(mallet_path, corpus, num_topics=10, id2word=corpus.dictionary)

#ldamallet = models.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

AttributeError: ignored

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']



In [ ]:
# Show
df_dominant_topics

In [ ]:
# Select the model and print the topics
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

# tunned mallet model

In [ ]:
from gensim import corpora, models, similarities
def convertldaGenToldaMallet(mallet_model):
    model_gensim = models.ldamodel.LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha, eta=0,
    )
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

optimal_model_1 = convertldaGenToldaMallet(optimal_model)

In [ ]:
import pickle 

In [ ]:
pickle.dump(optimal_model, open('optimal_lda_model_1', 'wb'))
pickle.dump(corpus, open('corpus', 'wb'))
pickle.dump(id2word, open('id2word', 'wb'))

In [ ]:

#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(optimal_model_1, corpus, id2word)
p

In [ ]:
p = pyLDAvis.gensim.prepare(optimal_model_1, corpus, id2word)
pyLDAvis.save_html(p, 'output_mallet_lda.html')

# Frequency Distribution of Word Counts in Documents

In [ ]:
doc_lens = [len(d) for d in df_dominant_topic.Text]

# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.hist(doc_lens, bins = 1000, color='navy')
plt.text(750, 100, "Mean   : " + str(round(np.mean(doc_lens))))
plt.text(750,  90, "Median : " + str(round(np.median(doc_lens))))
plt.text(750,  80, "Stdev   : " + str(round(np.std(doc_lens))))
plt.text(750,  70, "1%ile    : " + str(round(np.quantile(doc_lens, q=0.01))))
plt.text(750,  60, "99%ile  : " + str(round(np.quantile(doc_lens, q=0.99))))

plt.gca().set(xlim=(0, 1000), ylabel='Number of Documents', xlabel='Document Word Count')
plt.tick_params(size=16)
plt.xticks(np.linspace(0,1000,9))
plt.title('Distribution of Document Word Counts', fontdict=dict(size=22))
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.colors as mcolors
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

fig, axes = plt.subplots(2,2,figsize=(10,7), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):    
    df_dominant_topic_sub = df_dominant_topic.loc[df_dominant_topic.Dominant_Topic == i, :]
    doc_lens = [len(d) for d in df_dominant_topic_sub.Text]
    ax.hist(doc_lens, bins = 1000, color=cols[i])
    ax.tick_params(axis='y', labelcolor=cols[i], color=cols[i])
    sns.kdeplot(doc_lens, color="black", shade=False, ax=ax.twinx())
    ax.set(xlim=(0, 1000), xlabel='Document Word Count')
    ax.set_ylabel('Number of Documents', color=cols[i])
    ax.set_title('Topic: '+str(i), fontdict=dict(size=16, color=cols[i]))

fig.tight_layout()
fig.subplots_adjust(top=0.90)
plt.xticks(np.linspace(0,1000,9))
fig.suptitle('Distribution of Document Word Counts by Dominant Topic', fontsize=22)
plt.show()

# Word Counts of Topic Keywords- LDA MODEL

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_lemmatized for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(10,10), sharey=True) #, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()

# mallet model

In [ ]:
from collections import Counter
topics = optimal_model.show_topics(formatted=False)
data_flat = [w for w_list in data_lemmatized for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(12,10), sharey=True) #, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()

# EVERY TIME I RUN THIS CODE DIFF WORDS APPEAR...WHY???????

# the most discussed topics in the documents

In [ ]:
# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs, wordid_topics, wordid_phivalues = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=lda_model, corpus=corpus, end=-1)            

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])
dominant_topic_in_each_doc = df.groupby('Dominant_Topic').size()
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in lda_model.show_topics(formatted=False) 
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

In [ ]:
df_top3words

In [ ]:
from matplotlib.ticker import FuncFormatter

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=120, sharey=True)

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.5, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id == x, 'words'].values[0] ) #.values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax1.set_ylabel('Number of Documents')
ax1.set_ylim(0, 1000)

# Topic Distribution by Topic Weights
ax2.bar(x='index', height='count', data=df_topic_weightage_by_doc, width=.5, color='steelblue')
ax2.set_xticks(range(df_topic_weightage_by_doc.index.unique().__len__()))
ax2.xaxis.set_major_formatter(tick_formatter)
ax2.set_title('Number of Documents by Topic Weightage', fontdict=dict(size=10))

plt.show()

# t-SNE Clustering Chart

In [34]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 214 samples in 0.004s...
[t-SNE] Computed neighbors for 214 samples in 0.006s...
[t-SNE] Computed conditional probabilities for sample 214 / 214
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 55.043941
[t-SNE] KL divergence after 1000 iterations: -0.418639


NameError: ignored

In [ ]:
tsne_lda

In [ ]:
df_tse = pd.DataFrame({'Column1': tsne_lda[:, 0], 'Column2': tsne_lda[:, 1]})

In [ ]:
df_tse.head() 

In [ ]:
x,y = zip(*tsne_lda)
plt.scatter(x, y)

In [ ]:
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plt.scatter(df_tse.Column1, df_tse.Column2, color=mycolors[topic_num])

In [ ]:
# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
#plot.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1], color=mycolors[topic_num])
plt.scatter(df_tse.Column1, df_tse.Column2, color=mycolors[topic_num])
show(plot)

In [90]:
# Import Dataset
df_bert = pd.read_csv('fb_ads.csv') #, encoding='cp1252')

df_bert.head()

,ad_id,page_id,page_name,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,funding_entity,impressions_min,impressions_max,spend_min,spend_max,ad_url
0,422855775441062,1787487104836976,Plasnewydd & Roath Community News,If you live in Plasnewydd & you're self-isolat...,NaN,NaN,NaN,2021-02-15,NaN,Plasnewydd & Roath Community News,0,999,0,99,https://www.facebook.com/ads/library/?id=42285...
1,129870885665778,101558386889,Shelter Scotland,Shelter Scotland's Law & Debt advice project c...,blog.scotland.shelter.org.uk,NaN,Shelter Scotland,2021-02-15,NaN,Shelter Scotland,0,999,0,99,https://www.facebook.com/ads/library/?id=12987...
2,414306946330402,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=41430...
3,1123586888080157,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=11235...
4,1374708909543169,1855491614716590,Pure Planet,{{product.brand}},purepla.net,{{product.description}},{{product.name}},2021-02-15,NaN,Pure Planet Ltd,0,999,0,99,https://www.facebook.com/ads/library/?id=13747...


# creating new col with 4 party names

In [9]:
!pip install texthero

     |████████████████████████████████| 241 kB 6.9 MB/s 
     |████████████████████████████████| 1.5 MB 37.0 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [91]:
import texthero as hero

In [92]:
# cleaning
df_bert['page_name'] = hero.clean(df_bert['page_name'])

In [93]:
col         = 'page_name'
conditions  = [ df_bert[col].str.contains('liberal'),
                df_bert[col].str.contains('conservative'),
                df_bert[col].str.contains('labour')  ] 
choices     = [ "liberal", 'conservative', 'labour' ]
    
#df_fb_ads_norm["ad_party_name"] = np.select(conditions, choices, default=np.nan)
df_bert["ad_party_name"] = np.select(conditions, choices, default='other')

In [94]:
df_bert['ad_party_name'].astype('category').value_counts()

other           4274
labour           494
liberal          124
conservative     111
Name: ad_party_name, dtype: int64

In [95]:
#ad_party_name_three = df_bert['ad_party_name'].isin(['labour', 'liberal', 'conservative'])
#df_bert[~ad_party_name_three]
df_bert2 = df_bert.copy()
df_bert2.drop(df_bert2.index[df_bert2['ad_party_name'] == 'other'], inplace = True)

In [96]:
df_bert2.head()

,ad_id,page_id,page_name,ad_creative_body,ad_creative_link_caption,ad_creative_link_description,ad_creative_link_title,ad_delivery_start_time,ad_delivery_stop_time,funding_entity,impressions_min,impressions_max,spend_min,spend_max,ad_url,ad_party_name
54,432656598051836,149065065103747,wyre forest labour,My name is Jackie Griffiths. I have lived in W...,NaN,NaN,NaN,2021-02-15,NaN,Stephen Alan Brown,0,999,0,99,https://www.facebook.com/ads/library/?id=43265...,labour
114,427211031698393,804917826282199,elmbridge liberal democrats,Usually we'd be knocking on your door to find ...,NaN,NaN,NaN,2021-02-15,NaN,Jaska Alanko,0,999,0,99,https://www.facebook.com/ads/library/?id=42721...,liberal
115,446722996681758,804917826282199,elmbridge liberal democrats,In May we'll vote in three elections - Surrey ...,www.elmbridgelibdems.org.uk,NaN,Judy Sarsby - Lib Dem candidate in Weybridge R...,2021-02-15,NaN,Jaska Alanko,0,999,0,99,https://www.facebook.com/ads/library/?id=44672...,liberal
127,254624256168293,804917826282199,elmbridge liberal democrats,In May we'll vote in three elections - Surrey ...,www.elmbridgelibdems.org.uk,NaN,Sue Grose - Lib Dem candidate for Surrey Count...,2021-02-15,NaN,Jaska Alanko,0,999,0,99,https://www.facebook.com/ads/library/?id=25462...,liberal
129,189589009581352,1760421280675031,tynemouth constituency labour,{{product.brand}},postalvote.labour.org.uk,{{product.description}},{{product.name}},2021-02-15,NaN,Tynemouth Constituency Labour Party,2000,2999,0,99,https://www.facebook.com/ads/library/?id=18958...,labour


In [43]:
!pip install bertopic[visualization] --quiet

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 7.4 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 6.4 MB 54.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.2 MB 145 kB/s 
     |████████████████████████████████| 2.6 MB 40.8 MB/s 
     |████████████████████████████████| 1.2 MB 41.8 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 636 kB 54.3 MB/s 
     |████████████████████████████████| 3.3 MB 26.0 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 


In [ ]:
pip install bertopic[visualization]

In [ ]:
!pip3 install "bertopic[visualization]"

In [ ]:
import torch
print(torch.__version__)

In [14]:
!pip install bertopic

In [ ]:
!pip install bertopic==0.8.1

In [97]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [98]:
df_bert['ad_creative_body_str']=df_bert['ad_creative_body'].apply(str)

In [99]:
docs = df_bert.ad_creative_body_str.values.tolist()

In [100]:
import re
# Remove Emails
docs = [re.sub('\S*@\S*\s?', '', sent) for sent in docs]

# Remove new line characters
docs = [re.sub('\s+', ' ', sent) for sent in docs]

# Remove distracting single quotes
docs = [re.sub("\'", "", sent) for sent in docs]

print(docs[:1])

['If you live in Plasnewydd & youre self-isolating or struggling to get out, your local Councillors are here to help. Just email ']


In [ ]:
# Creating Topics

In [101]:
model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

In [102]:
topics, probs = model.fit_transform(docs)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2021-08-30 15:06:59,595 - BERTopic - Transformed documents to Embeddings
2021-08-30 15:07:14,521 - BERTopic - Reduced dimensionality with UMAP
2021-08-30 15:07:23,504 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
#extract most frequent topics

In [103]:
model.get_topic_freq()

,Topic,Count
0,0,510
1,-1,447
2,1,104
3,2,82
4,3,78
...,...,...
162,157,11
161,163,11
160,164,11
159,165,11


In [ ]:
#Get Individual Topics

In [104]:
model.get_topic(0)

[('productbrand', 0.8303548441728368),
 ('glands', 0.013172485386494607),
 ('biochemical', 0.013172485386494607),
 ('mednieks', 0.013172485386494607),
 ('labs', 0.013172485386494607),
 ('salivary', 0.013172485386494607),
 ('immunological', 0.013172485386494607),
 ('proteins', 0.013172485386494607),
 ('brands', 0.008731938585016143),
 ('factory', 0.0)]

In [ ]:
model.get_topic(1)

In [ ]:
#Visualize Topics

In [105]:
model.visualize_topics()

In [89]:
#model.visualize_distribution(probs[200])

In [107]:
model.visualize_hierarchy(top_n_topics=30)

In [109]:
model.visualize_barchart(top_n_topics=5)

In [111]:
model.visualize_barchart(top_n_topics=10)

In [112]:
model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

In [113]:
model.visualize_heatmap(n_clusters=10, width=1000, height=1000)

In [114]:
model.visualize_term_rank()

In [115]:
timestamps = df_bert.ad_delivery_start_time.to_list()

In [ ]:
#Topics over Time

In [116]:
topics_over_time = model.topics_over_time(docs=docs, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

19it [00:14,  1.28it/s]


In [ ]:
#Visualize Topics over Time

In [117]:
model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [118]:
model.visualize_topics_over_time(topics_over_time, top_n_topics=5)

In [33]:
model = BERTopic(nr_topics="auto")

In [54]:
classes = [df_bert["ad_party_name"][i] for i in df_bert["ad_party_name"]]

KeyError: ignored

In [ ]:



# Create topic model and calculate topics per class
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)
topics_per_class = topic_model.topics_per_class(docs, topics, classes=classes)

In [55]:
model.visualize_topics_per_class(topics_per_class)

NameError: ignored

In [119]:
topics_per_class = model.topics_per_class(docs, topics, classes=df_bert["ad_party_name"])

4it [00:08,  2.11s/it]


In [120]:
model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)

In [67]:
topics_per_class = model.topics_per_class(docs, topics, classes=df_bert["ad_party_name"])

4it [00:07,  1.98s/it]


In [121]:
model.visualize_topics_per_class(topics_per_class, top_n_topics=5, width=900)

In [ ]:
# excluding other party name analysis

In [122]:
df_bert2['ad_creative_body_str']=df_bert2['ad_creative_body'].apply(str)

In [123]:
docs = df_bert2.ad_creative_body_str.values.tolist()

In [124]:
import re
# Remove Emails
docs = [re.sub('\S*@\S*\s?', '', sent) for sent in docs]

# Remove new line characters
docs = [re.sub('\s+', ' ', sent) for sent in docs]

# Remove distracting single quotes
docs = [re.sub("\'", "", sent) for sent in docs]

print(docs[:1])

['My name is Jackie Griffiths. I have lived in Worcestershire my whole life; I was originally from Droitwich, before I moved to Stourport over 30 years ago, where as a working mom I raised my 3 children. I had a long and enjoyable career with HMRC, including senior leadership roles, which allowed me to develop my skills in management, human resources, and business delivery. In my retirement, I have become a school governor and in May 2019, I was delighted to be elected to Stourport Town Council. If you vote for me, I would bring energy, enthusiasm and fresh ideas on tackling problems and getting things done. In particular, I believe in supporting local jobs, affordable public transport and social care, and the Green New Deal. It would be a honour and privilege to work with and for the people of Stourport-on-Severn and Worcestershire, representing the issues which matter most to those who call this lovely county home.']


In [ ]:
# Creating Topics

In [125]:
model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

In [126]:
topics, probs = model.fit_transform(docs)

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

2021-08-30 15:15:06,499 - BERTopic - Transformed documents to Embeddings
2021-08-30 15:15:14,590 - BERTopic - Reduced dimensionality with UMAP
2021-08-30 15:15:14,661 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
#extract most frequent topics

In [128]:
model.get_topic_freq()

,Topic,Count
0,0,271
1,1,219
2,2,105
3,3,58
4,4,33
5,5,18
6,6,17
7,-1,8


In [ ]:
#Visualize Topics

In [129]:
model.visualize_topics()

In [130]:
model.visualize_hierarchy(top_n_topics=30)

In [131]:
model.visualize_barchart(top_n_topics=5)

In [132]:
timestamps = df_bert2.ad_delivery_start_time.to_list()

In [ ]:
#Topics over Time

In [133]:
topics_over_time = model.topics_over_time(docs=docs, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

9it [00:00,  9.52it/s]


In [ ]:
#Visualize Topics over Time

In [136]:
model.visualize_topics_over_time(topics_over_time, top_n_topics=15)

In [137]:
topics_per_class = model.topics_per_class(docs, topics, classes=df_bert2["ad_party_name"])

3it [00:00,  5.51it/s]


In [138]:
model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)